<a href="https://colab.research.google.com/github/vignesh99/SpeechTechnology-CS6300/blob/master/E2E_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
                                    #Get data files onto root
%cd
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/TIMIT\ E2E/timit.zip /root
!unzip /root/timit.zip -d /root
!cp -avr /content/gdrive/My\ Drive/TIMIT\ E2E/Wordset\ files/* /root
!cp -avr /content/gdrive/My\ Drive/TIMIT\ E2E/Wordceps\ files/* /root
!ls

Streaming output truncated to the last 5000 lines.
  inflating: /root/timit_train/fsgf0/sa1.PHN_SP  
  inflating: /root/timit_train/fsgf0/sa1.mfc  
  inflating: /root/timit_train/fsgf0/si2187.wrd  
  inflating: /root/timit_train/fsgf0/sa2.PHN_SP  
  inflating: /root/timit_train/fsgf0/sx387.PHN_SP  
  inflating: /root/timit_train/fsgf0/sx207.mfc  
  inflating: /root/timit_train/fsgf0/sx27.PHN_SP  
  inflating: /root/timit_train/fsgf0/si2187.mfc  
  inflating: /root/timit_train/fsgf0/si927.txt  
  inflating: /root/timit_train/fsgf0/sa2.wrd  
  inflating: /root/timit_train/fsgf0/si927.wrd  
 extracting: /root/timit_train/fsgf0/sa2.txt  
  inflating: /root/timit_train/fsgf0/sx387.wrd  
 extracting: /root/timit_train/fsgf0/sx207.txt  
  inflating: /root/timit_train/fsgf0/si2187.txt  
  inflating: /root/timit_train/fsgf0/sx27.phn  
  inflating: /root/timit_train/fsgf0/sx27.wav  
  inflating: /root/timit_train/fsgf0/si927.phn  
  inflating: /root/timit_train/fsgf0/si2187.PHN_SP  
  inflating:

In [2]:
                                    #Import libraries
from pylab import *
import os
from google.colab import files
!pip install pickle-mixin           #Use pkl to store model
from pickle import dump
from pickle import load      
!pip install SoundFile
import soundfile as sf              #For .wav to data
from scipy.io import wavfile
!pip install python_speech_features
from python_speech_features import mfcc
import pandas as pd
import csv

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-cp36-none-any.whl size=6000 sha256=bd5e7e6d10fff392b82a24892289154f846326dcdb3f522afa470e4b8de30b65
  Stored in directory: /root/.cache/pip/wheels/cd/05/42/71de70fa36b9cbb7657bb5793a16f8028c1cdc1bdd3b8e1ac3
Successfully built pickle-mixin
  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=b4d6a9ebb1c127d3dfc5a2432ff43298a8a48d957734402b7b46cbbc8f378984
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [ ]:
                                    #Import torch requirements
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
#words = load(open('words.pkl', 'rb'))
#inds = load(open('inds.pkl', 'rb'))
reps = load(open('reps.pkl', 'rb'))
wordset = load(open('wordset.pkl', 'rb'))

Nreps = 250							#Number of repititions of each word

inds = np.where(reps >= Nreps)[0]
words = wordset[inds]               #Choose only 36-most-repeated words
freqs = reps[inds]                  #Frequency at which these words occur
print(len(inds))
print(words)
print(freqs)

27
['a' 'all' 'an' 'and' 'ask' 'carry' 'dark' "don't" 'greasy' 'had' 'in'
 'is' 'like' 'me' 'of' 'oily' 'rag' 'she' 'suit' 'that' 'the' 'to' 'wash'
 'water' 'year' 'you' 'your']
[ 866  543  571  481  464  463  465  488  462  526  944  399  511  505
  454  470  470  571  462  611 1602 1018  469  479  472  271  565]


In [4]:
def mostrepeatedwords(Nrep=250) :
    allwords = []
                                    #Go through all folders and consolidate the words
    for subdir, dirs, files in os.walk("timit_train"):
        dirname = subdir.split(os.path.sep)[-1]
        print('Directory:', dirname)
        for f in files:
            if f.endswith('.txt'):
                                    #Get words of each sentence from .txt file
                !cp -avr /root/timit_train/$dirname/$f /root
                words = np.genfromtxt(f,delimiter=' ',dtype=str)
                words = words[2:]
                words[-1] = words[-1][:-1]
                                    #Put all the words in a list
                allwords += list(map(str.lower,list(words)))
                !rm $f
                                    #Get the unique set of words and their counts
    wordunique,wordcount = np.unique(allwords,return_counts=True)
                                    #Choose only those words with significant repitions
    indrep = np.where(wordcount >= Nrep)[0]
    #print(len(indrep))
    wordclass = wordunique[indrep]
    return wordclass,wordunique,wordcount,indrep

In [5]:
# words,wordset,reps,inds = mostrepeatedwords()
# words

In [6]:
                                    #Dump words model file
# dump(words, open('words.pkl', 'wb'))
# dump(inds, open('inds.pkl', 'wb'))
# dump(reps, open('reps.pkl', 'wb'))
# dump(wordset, open('wordset.pkl', 'wb'))

In [7]:
                                    #Download model files
# files.download("/root/words.pkl")
# files.download("/root/inds.pkl")
# files.download("/root/reps.pkl")
# files.download("/root/wordset.pkl")

In [9]:
                                    #Getting the mfcc features for different words
def wavtoMfcc(words,Nmfcc=38) :
    count = 0                       #Initialize required lists
    timit_audio = []
    new_timit_audio = []
    timitceps = []
    wordloop = []
    lenframes = []
    for subdir, dirs, files in os.walk('timit_train'):
        dirname = subdir.split(os.path.sep)[-1]
        for f in files:
            if f.endswith('.wav'):
                count += 1
                print(os.path.join(subdir, f), ', name:', f, ', Count:', count)
                                    #Read data from the .wav file
                data, fs = sf.read(os.path.join(subdir, f))
                timit_audio.append(data)
                fword = f[:-4]+".wrd"
                                    #Check word files for most repeated words
                print(os.path.join(subdir, fword), ', name:', fword, ', Count:', count)
                word_file = np.array(pd.read_csv(os.path.join(subdir, fword), delimiter = ' ',header = None))
                new_data = []
                for w in np.array(word_file[:,2]):
                    if w in words:
                        a = np.where(word_file[:,2] == w)
                        start = word_file[a[0],0][0]
                        end = word_file[a[0],1][0]
                        if(start != end):   #For removing corner case error
                                    #Collect the .wav samples of required slices
                            new_data = data[start:end]
                            new_timit_audio.append(new_data)
                                            #Convert them to mfcc
                            ceps = mfcc(new_data, fs, numcep=Nmfcc,winlen=0.008,winstep=0.004,nfilt=76)
                            timitceps.append(ceps)
                                    #Put the words in order of the mfcc features
                            wordloop.append(w)
                            lenframes.append(len(ceps))
    
    timitceps = array(timitceps)
    wordloop = array(wordloop)
    lenframes = array(lenframes) 
    maxframes = lenframes.max()     #Obtain maximum size of frames

    return timitceps,wordloop,lenframes,maxframes,new_timit_audio                                           

In [10]:
# timitceps,wordloop,lenframes,maxframes,new_timit_audio = wavtoMfcc(words)

In [11]:
                                    #Dump processed files
# dump(timitceps, open('timitceps.pkl', 'wb'))
# dump(wordloop, open('wordloop.pkl', 'wb'))
# dump(lenframes, open('lenframes.pkl', 'wb'))
# dump(new_timit_audio, open('new_timit_audio.pkl', 'wb'))

In [12]:
                                    #Download model files
# files.download("/root/timitceps.pkl")
# files.download("/root/wordloop.pkl")
# files.download("/root/lenframes.pkl")
# files.download("/root/new_timit_audio.pkl")

In [13]:
                                    #Load the required files
timitceps = load(open('timitceps.pkl', 'rb'))
wordloop = load(open('wordloop.pkl', 'rb'))
lenframes = load(open('lenframes.pkl', 'rb'))
# new_timit_audio = load(open('new_timit_audio.pkl', 'rb'))

In [57]:
def mfcctotensor(words,trainfreqs,testfreqs,timitceps,wordloop,lenframes,maxframes=162,Nmfcc=38) :
                                    #Declare train and test arrays
    Xtrain = np.zeros((sum(trainfreqs),maxframes,Nmfcc))
    Xtest = np.zeros((sum(testfreqs),maxframes,Nmfcc))
    Ytrain = np.zeros(sum(trainfreqs))
    Ytest = np.zeros(sum(testfreqs))
    indtrain = cumsum(trainfreqs)   #Cumulative sum of indices
    indtest = cumsum(testfreqs)
                                    #Loop to collect train and test features
    for i in range(0,len(words)) :
    
                                    #Obtain the particular word's mfcc features
        indcep = np.where(wordloop==words[i])[0]
        indlen = lenframes[indcep]
                                    #Can add randomness if needed (while choosing for train and test)
        trainindcep = indcep[:trainfreqs[i]]
        testindcep = indcep[trainfreqs[i]:]
        trainindlen = indlen[:trainfreqs[i]]
        testindlen = indlen[trainfreqs[i]:]
                                    #Check if occurance match
        if trainfreqs[i] + testfreqs[i] == len(indcep) :            
                                    #Loop over train features and transfer from list to array
            for j in range(0,len(trainindcep)) :
                if i != 0 :
                    Xtrain[indtrain[i-1]+j][:trainindlen[j]] = timitceps[trainindcep[j]]
                else :
                    Xtrain[j][:trainindlen[j]] = timitceps[trainindcep[j]]
                                    #Loop over test features and transfer from list to array
            for j in range(0,len(testindcep)) :
                if i != 0 :
                    Xtest[indtest[i-1]+j][:testindlen[j]] = timitceps[testindcep[j]]
                else :
                    Xtest[j][:testindlen[j]] = timitceps[testindcep[j]]
        else :
            print("ERROR : Frequencies don't match with index length")
            return None
                                    #Assign outputs for ANN
        if i == 0 :
            Ytrain[:indtrain[i]] = i
            Ytest[:indtest[i]] = i
        else :
            Ytrain[indtrain[i-1]:indtrain[i]] = i
            Ytest[indtest[i-1]:indtest[i]] = i

    return Xtrain,Ytrain,Xtest,Ytest,words,trainfreqs,testfreqs

In [58]:
def traintesttensors(timitceps,wordloop,lenframes,Nmfcc=38) :
                                    #Get required inputs
    words,freqs = np.unique(wordloop,return_counts=True)
    trainfreqs = np.round(0.8*freqs.astype("float"))
    testfreqs = freqs-trainfreqs
    maxframes = lenframes.max()

    return mfcctotensor(words,trainfreqs.astype("int"),testfreqs.astype("int"),timitceps,wordloop,lenframes,maxframes)

In [59]:
Xtrain,Ytrain,Xtest,Ytest,words,trainfreqs,testfreqs = traintesttensors(timitceps,wordloop,lenframes)

In [64]:
                                    #Dump processed files
# dump(Xtrain, open('Xtrain.pkl', 'wb'))
# dump(Ytrain, open('Ytrain.pkl', 'wb'))
# dump(Xtest, open('Xtest.pkl', 'wb'))
# dump(Ytest, open('Ytest.pkl', 'wb'))
# dump(words, open('words.pkl', 'wb'))
# dump(trainfreqs, open('trainfreqs.pkl', 'wb'))
# dump(testfreqs, open('testfreqs.pkl', 'wb'))

In [65]:
                                    #Download model files
# files.download("/root/Xtrain.pkl")
# files.download("/root/Ytrain.pkl")
# files.download("/root/Xtest.pkl")
# files.download("/root/Ytest.pkl")
# files.download("/root/words.pkl")
# files.download("/root/trainfreqs.pkl")
# files.download("/root/testfreqs.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
print(type(timitceps[0]))
_,check = array(np.unique(wordloop,return_counts=True))
print(sum(np.round(0.8*check.astype(float))))#/sum(check.astype(float)))
(0.8*check.astype(float))
sum(check.astype(float))
a = array([2,3,4,5,6])
cumsum(a)

<class 'numpy.ndarray'>
12520.0


array([ 2,  5,  9, 14, 20])

In [ ]:

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,units=256):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.units = units

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size,num_layers=self.units)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,units=256):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.units = units

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size,num_layers=self.units)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH,units=256):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.units = units

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size,num_layers=self.units)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
a = np.where(wordloop=="carry")[0]
shape(timitceps[a[5]])
#wordloop[2]

(73, 38)

In [ ]:
wordloop[np.where(lenframes==lenframes.min())[0]]
#wavfile.write('test.wav', 16000, new_timit_audio[15545])

array(['to', 'a'], dtype='<U6')

In [ ]:
print(lenframes.max())
lenframes.min()

162


3

In [ ]:
print(words)
print(freqs)

['a' 'all' 'an' 'and' 'are' 'ask' 'be' 'carry' 'dark' "don't" 'for'
 'greasy' 'had' 'he' 'his' 'in' 'is' 'like' 'me' 'of' 'oily' 'on' 'rag'
 'she' 'suit' 'that' 'the' 'to' 'was' 'wash' 'water' 'we' 'with' 'year'
 'you' 'your']
[ 866  543  571  481  237  464  175  463  465  488  216  462  526  233
  189  944  399  511  505  454  470  164  470  571  462  611 1602 1018
  235  469  479  154  187  472  271  565]


In [ ]:
files.download("test.wav")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>